In [1]:
import os
from osgeo import gdal, ogr, osr
import numpy as np

In [18]:
raster_dir = '../../Landsat_ARD_002008/LE08'

for folder in os.listdir(raster_dir):
    folder_path = os.path.join(raster_dir, folder, 'SR')
    file_list = os.listdir(folder_path)
    for file in file_list:
        if file.endswith('SRB1.tif'):
            CAerosol_path = os.path.join(folder_path, file)
        elif file.endswith('SRB2.tif'):
            BLUE_path = os.path.join(folder_path, file)
        elif file.endswith('SRB3.tif'):
            GREEN_path = os.path.join(folder_path, file)
        elif file.endswith('SRB4.tif'):
            RED_path = os.path.join(folder_path, file)
        elif file.endswith('SRB5.tif'):
            NIR_path = os.path.join(folder_path, file)
        elif file.endswith('SRB6.tif'):
            SWIR1_path = os.path.join(folder_path, file)
        elif file.endswith('SRB7.tif'):
            SWIR2_path = os.path.join(folder_path, file)
    merged_path = os.path.join(raster_dir, file[:-26])
    os.system('gdalbuildvrt -separate ' + merged_path + '.vrt ' + CAerosol_path + ' ' + BLUE_path + ' ' + GREEN_path + ' ' + RED_path + ' ' + NIR_path + ' ' + SWIR1_path + ' ' + SWIR2_path)
    os.system('gdal_translate ' + merged_path + '.vrt ' + merged_path + '.tif')

In [3]:
raster_dir = '../../Landsat_ARD_002008/LE08/20210625/SR'
shp_path = '../../Sac_Yolo_Solano_Boundaries.shp'
output_path = '../data/Data'
#  -crop_to_cutline -dstalpha 
for file in os.listdir(raster_dir):
    if file.endswith('SRB1.tif'):
        os.system('gdalwarp -cutline ' + shp_path + ' ' + os.path.join(raster_dir, file) + ' ' + os.path.join(raster_dir, file[:-4] + '_clip.tif'))
        raster = gdal.Open(os.path.join(raster_dir, file[:-4] + '_clip.tif'))
        CAerosol = raster.GetRasterBand(1).ReadAsArray()
    elif file.endswith('SRB2.tif'):
        os.system('gdalwarp -cutline ' + shp_path + ' ' + os.path.join(raster_dir, file) + ' ' + os.path.join(raster_dir, file[:-4] + '_clip.tif'))
        raster = gdal.Open(os.path.join(raster_dir, file[:-4] + '_clip.tif'))
        BLUE = raster.GetRasterBand(1).ReadAsArray()
    elif file.endswith('SRB3.tif'):
        os.system('gdalwarp -cutline ' + shp_path + ' ' + os.path.join(raster_dir, file) + ' ' + os.path.join(raster_dir, file[:-4] + '_clip.tif'))
        raster = gdal.Open(os.path.join(raster_dir, file[:-4] + '_clip.tif'))
        GREEN = raster.GetRasterBand(1).ReadAsArray()
    elif file.endswith('SRB4.tif'):
        os.system('gdalwarp -cutline ' + shp_path + ' ' + os.path.join(raster_dir, file) + ' ' + os.path.join(raster_dir, file[:-4] + '_clip.tif'))
        raster = gdal.Open(os.path.join(raster_dir, file[:-4] + '_clip.tif'))
        RED = raster.GetRasterBand(1).ReadAsArray()
    elif file.endswith('SRB5.tif'):
        os.system('gdalwarp -cutline ' + shp_path + ' ' + os.path.join(raster_dir, file) + ' ' + os.path.join(raster_dir, file[:-4] + '_clip.tif'))
        raster = gdal.Open(os.path.join(raster_dir, file[:-4] + '_clip.tif'))
        NIR = raster.GetRasterBand(1).ReadAsArray()
    elif file.endswith('SRB6.tif'):
        os.system('gdalwarp -cutline ' + shp_path + ' -crop_to_cutline -dstalpha ' + os.path.join(raster_dir, file) + ' ' + os.path.join(raster_dir, file[:-4] + '_clip.tif'))
        raster = gdal.Open(os.path.join(raster_dir, file[:-4] + '_clip.tif'))
        SWIR1 = raster.GetRasterBand(1).ReadAsArray()
    elif file.endswith('SRB7.tif'):
        os.system('gdalwarp -cutline ' + shp_path + ' ' + os.path.join(raster_dir, file) + ' ' + os.path.join(raster_dir, file[:-4] + '_clip.tif'))
        raster = gdal.Open(os.path.join(raster_dir, file[:-4] + '_clip.tif'))
        SWIR2 = raster.GetRasterBand(1).ReadAsArray()

print(CAerosol.shape)
print(BLUE.shape)
raster_array = np.dstack((CAerosol, BLUE, GREEN, RED, NIR, SWIR1, SWIR2))

raster = gdal.Open(os.path.join(raster_dir, file[:-4] + '_clip.tif'))
geotransform = raster.GetGeoTransform()

output = os.path.join('../data/Data', file[:-26] + '.tif')

driver = gdal.GetDriverByName('GTiff')
outRaster = driver.Create(output, raster.RasterXSize, raster.RasterYSize, raster_array.shape[2], gdal.GDT_Int16)
outRaster.SetGeoTransform((geotransform[0], geotransform[1], 0, geotransform[3], 0, geotransform[5]))
outRaster.SetProjection(raster.GetProjection())
for i in range(raster_array.shape[2]):
    outband = outRaster.GetRasterBand(i + 1)
    outband.WriteArray(raster_array[:, :, i])
outRaster.FlushCache()  # write to disk
outRaster = None
# outRasterSRS = osr.SpatialReference()
# outRasterSRS.ImportFromWkt(raster.GetProjectionRef())
# outRaster.SetProjection(outRasterSRS.ExportToWkt())

(5000, 5000)


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 5000 and the array at index 5 has size 3751

In [9]:
shp_path = '../../Sac_Yolo_Solano_Extent.shp'
output_clip = os.path.join('../data/Data', file[:-26] + '_clip.tif')
os.system('gdalwarp -cutline ' + shp_path + ' ' + output + ' ' + output_clip)

label_raster = '../data/GroundTruth/CDL_2021_06.tif'
label_raster_clip = label_raster[:-4] + '_clip.tif'
os.system('gdalwarp -cutline ' + shp_path + ' -crop_to_cutline -dstalpha ' + label_raster + ' ' + label_raster_clip)

1

In [ ]:
gdalwarp -cutline INPUT.shp -crop_to_cutline -dstalpha INPUT.tif OUTPUT.tif

In [ ]:
os.system('gdalwarp -overwrite -t_srs "' + epsg_code + '" -tr 30 30 -r near -te ' + boundStr + ' "' + current_file_remote + '" "' + export_file_remote + '"')